# EE123 Lab 2: Software Defined Radio -  Part I

### Written by Miki Lustig

Updated by Drake Lin, Spring 2023

Updated by Josh Sanz, Spring 2020, Spring 2022

Updated by Alan Dong, Spring 2021

## Frequency analysis using the DFT and windowing

In this lab you will reproduce the frequency analysis slides demonstrated in class, and look at the effect of using different windows, length of sequence, and averaging.

In [1]:
# Import functions and libraries
import numpy as np, matplotlib.pyplot as plt
from numpy import *
from numpy.fft import *
from matplotlib.pyplot import *

import numpy.fft as fft

%matplotlib inline

The following code implements the windowed DFTs that were shown in the lecture:

In [98]:
def pltWinFFTs(y, N, window_sig, name):
    # plots Individual FFTs
    #
    #  y -  Signal
    #  N -  Length of DFT (zero-padding)
    #  window_sig - Window
    #  name - Name of window
    
    window = fft.fft(y*window_sig, N)
    window_norm = 20*log10(abs(window)/max(abs(window)))
    fig = figure(figsize=(16,2))
    plot(r_[0.0:N/2.0]/N*2*pi,window_norm[0:N//2])
    plt.title(f'{name}')
    plt.ylabel('dB')
    plt.xlabel('rad/samp')
    axis((0,pi,-80,2))
    show()
    close()
    

def pltWindowedFFT(y, N):
    # pltWindowedFFT takes a signal y, multiplies it with several windows,
    # computes the N-length DFT of the (zero-padded) sequence,
    # and displays its single sided spectrum.
    #
    #  y -  Signal
    #  N -  Length of DFT (zero-padding)
    
    pltWinFFTs(y, N, np.ones(L), "Rect")
    pltWinFFTs(y, N, np.bartlett(L), "Triangle")
    pltWinFFTs(y, N, np.hanning(L), "Hann")
    pltWinFFTs(y, N, np.hamming(L), "Hamming")
    pltWinFFTs(y, N, np.blackman(L), "Blackman")
    pltWinFFTs(y, N, np.kaiser(L, 6), "Kaiser Beta=6")
    pltWinFFTs(y, N, np.kaiser(L, 9), "Kaiser Beta=9")

Here we generate a 128 length sequence. The sequence is a sum of sinusoids that are close in frequency, but have a large dynamic range in the amplitude. We first compute the DFT of the sequence with different windows. The different window functions will trade-off the mainlobe width with sidelobe spectral artifacts. 

In [99]:
noise_std = 0 # standard deviation of additive noise
N = 128       # FFT size
L = 128       # Window size

n = r_[0.0:L]

omega1 = 2*pi*0.2   # frequency 1
omega2 = 2*pi*0.25  # frequency 2
A1 = 1              # amplitude of first sinusoid
A2 = 0.005          # amplitude of second sinusoid
y = A1*sin(omega1*n) + A2*sin(omega2*n) + np.random.randn(L)*noise_std

pltWindowedFFT(y, N)

## Task 1: zero-padding

As you can see, the spectrum does not look "nice". In order to evaluate the spectrum at more points, in a way that is closer to how the DTFT would look, we need to zero-pad. 

* **Zero pad the sequence to 256.** Can you see improvement? Did the peak width or the sidelobe amplitudes change?
* **Zero pad the sequence to 512, 1024, 4096**. Will increasing the zero-padding further change the shape of the spectrum? 

Run each experiment in a different cell so you can see the results better. 

In [100]:
# Your code here



**Your answers here:**



In [101]:
# Your code here



In [102]:
# Your code here



In [103]:
# Your code here



**Your answers here:**



## Task 2: Window sizes and noise

In this task we will add Gaussian noise with standard deviation of 0.01 to the sequence. We will look at how increasing the sequence length improves the spectral resolution, as well as the signal-to-noise ratio of the spectrum.

**Set the DFT length to N=4096, and set noise_std=0.01**

* **Generate the sequence for L=128 and plot the windowed spectra.** Can you see the small peak? 
* **Generate the sequence for L=512 and plot the windowed spectra.** Can you see the small peak? Comment on the resolution of the peaks.
* **Generate the sequence for L=2048 and plot the windowed spectra.** Comment on the signal-to-noise ratio. Comment on the resolution of the peaks.

In general, for stationary signals, increasing the size of the window by a factor of $P$ will result in improved SNR of $\sqrt{P}$

In [104]:
# Generate the sequence for L=128 and plot the windowed spectra. Can you see the small peak? 
# Your code here:



**Your answers here:**



In [105]:
# Generate the sequence for L=512 and plot the windowed spectra. Can you see the small peak?
# Comment on the resolution of the peaks.
# Your code here:



**Your answers here:**



In [106]:
# Generate the sequence for L=2048 and plot the windowed spectra. 
# Comment on the signal-to-noise ratio. Comment on the resolution of the peaks.
# Your code here:



**Your answers here:**



## Task 3: Average Power Spectrum

As we increase the size of the window, the SNR of our spectrum improves, and the resolution of the spectrum also improves. 
However, sometimes we are not necessarily interested in increasing the resolution or size of our FFTs, but we are interested in improving SNR. 

One way to do so is to break the signal into non-overlapping sections of the desired length, compute their magnitude spectra, and then average the results. 
This is often called the average power spectrum. 
It is important to note that averaging the complex spectrum will often not result in your desired outcome. The reason is that the phase of the peaks in each segment may be different, depending on how we split the signal and what frequencies are involved. 
In that case, complex averaging will probably lead to cancellation of the signal.


* **Set the DFT length to N=4096, and set noise_std=0.01. Generate the sequence with total length of: 10240.** 
* **Break the sequence into 20 segments of L=512. Window each segment with a Kaiser window with $\beta=6$.**
* **Zero-pad each segment to length N=4096 and compute the FFT.**
* **Compute the average magnitude of the spectra, and display the result using the function ``semilogy``**
* For comparison, **compute the average of the complex spectra and then take the magnitude and display the result.**
* **Compare the result of the average power spectrum to the spectrum of the Kaiser window in Task 2.**


In [107]:
# Generate noisy sines, break into segments, window, and compute the FFT.
# Your code here:

# End of your code

# Compute means
Pspect = mean(abs(X),axis=0)
Pspect_cplx = abs(mean(X,axis=0))

# Plotting:
fig = figure(figsize=(16,2))
semilogy(r_[0.0:N/2.0]/N*2*pi,Pspect[0:N//2])
plt.xlabel('rad/samp')
plt.title('Average power spectrum')
fig = figure(figsize=(16,2))
semilogy(r_[0.0:N/2.0]/N*2*pi,Pspect_cplx[0:N//2])
plt.xlabel('rad/samp')
title('Complex average of the spectrum');

**Your answers here:**

